## Bounding Boxes Convolution Network Training
I love buzzwords.


In [1]:
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
import cv2

Using Theano backend.


In [2]:
%load_ext autoreload
%autoreload 2
import sys
paths = ['../models', '../dataset', '../util']
for path in paths:
    if path not in sys.path:
        sys.path.append(path)
 
from bounding_boxes import *
from data_generators import get_data_with_bbox_coords
import dataset
from callbacks import ShowSegmentation

bboxes = get_bounding_boxes('../data')
bboxes = largest_bbox_per_image(bboxes)
train_data = dataset.train
val_data = dataset.validation
 
train_gen = get_data_with_bbox_coords(train_data, '../data/train', bboxes)
val_gen = get_data_with_bbox_coords(val_data, '../data/train', bboxes)
n_labels = len(dataset.labels)

Function to define the layers of the convnet.

In [3]:
def keras_model_conv():
    model = Sequential()
    
    model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu', input_shape=(3, 256, 256)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4))

    model.compile(loss='mse',
              optimizer=Adam(),
              metrics=['accuracy', 'mse'])

    return model

Initialization of the model.

In [4]:
model = keras_model_conv()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 16, 256, 256)  448         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 16, 128, 128)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 16, 128, 128)  0           maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 128, 128)  9280        dropout_1[0][0]                  
___________________________________________________________________________________________

Fit the model using a generator.

Use: 
fit_generator(self, generator, samples_per_epoch, nb_epoch, verbose=1, callbacks=None, validation_data=None, nb_val_samples=None, class_weight=None, max_q_size=10, nb_worker=1, pickle_safe=False, initial_epoch=0)


In [5]:
model.fit_generator(train_gen, samples_per_epoch=len(train_data),
                    nb_epoch=1, verbose=1,
                    callbacks=None,
                    validation_data=val_gen, nb_val_samples=len(val_data))

Epoch 1/1
3008/3021 [============================>.] - ETA: 2s - loss: 1333.3250 - acc: 0.5492 - mean_squared_error: 1333.3250  

/Users/pieter/anaconda2/envs/medim/lib/python2.7/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


3040/3021 [==============================] - 677s - loss: 1332.3347 - acc: 0.5493 - mean_squared_error: 1332.3347 - val_loss: 4049.1005 - val_acc: 0.7070 - val_mean_squared_error: 4049.1005
